<a href="https://colab.research.google.com/github/TheDataFestAI/Learning_Resources/blob/main/learning_poc/poc1_ml_clv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# POC for CLV (Customer Life Cycle Value)

## Reference:

1. Domain Knowledge:
    1. https://www.analyticsvidhya.com/blog/2020/10/a-definitive-guide-for-predicting-customer-lifetime-value-clv/
    2. https://towardsdatascience.com/data-driven-growth-with-python-part-3-customer-lifetime-value-prediction-6017802f2e0f
    3. https://www.kaggle.com/code/shailaja4247/customer-lifetime-value-prediction

# Step 1: Get the source data from Kaggle

Ref: https://github.com/TheDataFestAI/Learning_Resources/blob/main/learning_poc/download_data_from_kaggle.ipynb

In [4]:
# install the required packages
!pip install -q opendatasets

In [5]:
# upload the kaggle.json file into google colab
from google.colab import files

kaggle_filename = "kaggle.json"
kaggle_key = files._upload_file(filepath=kaggle_filename)

In [6]:
# download the desired kaggle dataset
# **it will not ask for kaggle user and access token if you have downloaded the kaggle.json in current location
import opendatasets as od

kaggle_dataset_url = 'https://www.kaggle.com/datasets/sergeymedvedev/customer_segmentation'
od.download(kaggle_dataset_url)

100%|██████████| 7.20M/7.20M [00:00<00:00, 80.2MB/s]

# Step 2: Load the data into pandas dataframe

## Get the Encoding of the file

In [6]:
# # helpful character encoding module
# import charset_normalizer

# with open('/content/customer_segmentation/customer_segmentation.csv', "rb") as raw_data:
#     result = charset_normalizer.detect(raw_data.read())

# print("encoding is {}".format(result))

```python
encoding is {'encoding': 'windows-1250', 'language': 'English', 'confidence': 1.0}
```

## Loading into dataframe

In [7]:
import pandas as pd

df = pd.read_csv('/content/customer_segmentation/customer_segmentation.csv',
                 encoding="windows-1250")

print(f"dataframe loaded: {df.shape}")

dataframe loaded: (541909, 8)


## fetch few records from the dataframe

In [8]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Step 3: Analysis the Data

## get column types

In [75]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

## check the unique values in dataframe per columns

In [9]:
df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

## get non-null count and datatype of each column

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## filter out the records which have null CustomerID

In [34]:
# # method 1: filter using `dataframe method`
# df[(~df['CustomerID'].notnull())]  # 135080 records
# df[(~df['CustomerID'].notnull()) | (~df['Description'].notna())]  # 135080 records
# df[(~df['CustomerID'].notnull()) & (~df['Description'].notna())]  # 1454  records


# # method 2: filter using `loc method`
# df.loc[(~df['CustomerID'].notnull())]  # 135080 records
# df.loc[(~df['CustomerID'].notnull()) | (~df['Description'].notna())]  # 135080 records
# df.loc[(~df['CustomerID'].notnull()) & (~df['Description'].notna())]  # 1454  records


# # method 2: filter using `query method`
# df.query("CustomerID != CustomerID")  # 135080 records
# df.query('CustomerID != CustomerID or Description != Description')  # 135080 records
# df.query('CustomerID != CustomerID and Description != Description')  # 1454  records

In [37]:
df.loc[(~df['CustomerID'].notnull()) | (~df['Description'].notna())].tail(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541531,581498,84997d,CHILDRENS CUTLERY POLKADOT PINK,1,12/9/2011 10:26,8.29,NaN,United Kingdom
541532,581498,85038,6 CHOCOLATE LOVE HEART T-LIGHTS,1,12/9/2011 10:26,4.13,NaN,United Kingdom
541533,581498,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,1,12/9/2011 10:26,16.63,NaN,United Kingdom
541534,581498,85049a,TRADITIONAL CHRISTMAS RIBBONS,5,12/9/2011 10:26,3.29,NaN,United Kingdom
541535,581498,85049e,SCANDINAVIAN REDS RIBBONS,4,12/9/2011 10:26,3.29,NaN,United Kingdom
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom
541540,581498,DOT,DOTCOM POSTAGE,1,12/9/2011 10:26,1714.17,NaN,United Kingdom


## get numeric data distribution

In [11]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [69]:
df['InvoiceNo'].value_counts()

573585     1114
581219      749
581492      731
580729      721
558475      705
           ... 
554023        1
554022        1
554021        1
554020        1
C558901       1
Name: InvoiceNo, Length: 25900, dtype: int64

## get duplicates data

In [55]:
df.groupby(['InvoiceNo', 'StockCode'])['InvoiceNo'].count().sort_values(ascending=False).head(20)

InvoiceNo  StockCode
555524     22698        20
C544580    S            16
555524     22697        12
572861     22775         8
C553531    S             7
572344     M             7
578289     23395         7
C544583    S             7
538514     21756         6
541266     21755         6
           21754         6
C558347    S             6
574481     23084         6
540524     21756         6
579146     22111         5
           23356         5
575753     47422         5
571039     21034         5
550331     85202         5
537646     22585         5
Name: InvoiceNo, dtype: int64

In [62]:
df.loc[(df['InvoiceNo'] == '555524') & (df['StockCode'] == '22698')].head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
213176,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,6/5/2011 11:37,2.95,16923.0,United Kingdom
213177,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,6/5/2011 11:37,2.95,16923.0,United Kingdom
213183,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,6/5/2011 11:37,2.95,16923.0,United Kingdom
213191,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,6/5/2011 11:37,2.95,16923.0,United Kingdom
213192,555524,22698,PINK REGENCY TEACUP AND SAUCER,1,6/5/2011 11:37,2.95,16923.0,United Kingdom


# Step 4: Data Cleaning inside Dataframe

## first, take a copy of the dataframe

1. on the copied dataframe, we can do the cleaning and can generate cleaned dataframe.
2. lastly we can compare our cleaned dataframe with the early unchanged dataframe.

In [38]:
from copy import deepcopy

cleaned_df = deepcopy(df)
print(f"df: {id(df)} and cleaned_df: {id(cleaned_df)} are not pointing to same memory")

df: 136803761212192 and cleaned_df: 136803286176224 are not pointing to same memory


## Drop columns

1. These columns not required for building the ML models.
2. Re-engineering or feature engineering to reduce the feature columns. This will improve the model accuracy.

In [40]:
# cleaned_df.drop(['Description'], axis=1)
cleaned_df.drop(columns=['Description'], inplace=True)

## Drop duplicates

In [60]:
cleaned_df.drop_duplicates(subset=['InvoiceNo', 'StockCode'], keep="last", inplace=True)

In [64]:
# check the duplcates record in cleaned dataframe
cleaned_df.loc[(df['InvoiceNo'] == '555524') & (df['StockCode'] == '22698')].head(5)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
213208,555524,22698,1,6/5/2011 11:37,2.95,16923.0,United Kingdom


## drop null records

In [71]:
cleaned_df.dropna(axis=0, how="any", inplace=True)

## Change the column data types

In [73]:
cleaned_df["CustomerID"] = cleaned_df["CustomerID"].apply(lambda x: int(x))  # float to int
cleaned_df["InvoiceDate"] = pd.to_datetime(cleaned_df["InvoiceDate"])  # object to datetime

## calcculate CLV and store into new column

## Final, cleaned dataframe:

In [77]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396681 entries, 0 to 541908
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    396681 non-null  object        
 1   StockCode    396681 non-null  object        
 2   Quantity     396681 non-null  int64         
 3   InvoiceDate  396681 non-null  datetime64[ns]
 4   UnitPrice    396681 non-null  float64       
 5   CustomerID   396681 non-null  int64         
 6   Country      396681 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 24.2+ MB


In [76]:
cleaned_df.head(5)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


# Step 5: Data Separation as X and y

1. X is set of feature columns
2. y is predicted column

In [ ]:
# y = cleaned_df["CLV"]

In [ ]:
# X = cleaned_df.loc[:, ['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']]

# Step 6: Split the dataset into Train and test

# Step 7: Create the Model

# Use Colab Secret

In [ ]:
# from google.colab import userdata
# userdata.get('kaggle_auth')